In [1]:
from __future__ import print_function, division
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision.models as models

%matplotlib inline

In [2]:
'''
import urllib.request
urllib.request.urlretrieve("http://www.yurikoz.com/new_train.npy", "new_train.npy")
data = np.load('new_train.npy')
classes = np.unique(data[:,1])
np.save('classes.npy', classes)
train_size = len(data) * 7 // 10
np.save('train.npy', data[:train_size])
np.save('test.npy', data[train_size:])
'''
print()

In [3]:
train = np.load('train.npy')
test = np.load('test.npy')
print(len(train), len(test))

116522 49938


In [4]:
classes = np.load('classes.npy')

In [5]:
class HieroglyphsDataset(Dataset):
    
    def __init__(self, files, transform=None):
        self.frame = np.vstack(([np.load(f) for f in files]))
        self.transform = transform

    def __len__(self):
        return len(self.frame)

    def __getitem__(self, idx):
        sample = {'image': self.frame[idx][0], 'category': np.searchsorted(classes, self.frame[idx][1])}
        if self.transform:
            sample = self.transform(sample)
        return sample

In [6]:
class Trans(object):
    def __call__(self, sample):
        image, category = sample['image'], sample['category']
        t = torch.from_numpy(image).view(-1,image.shape[0],image.shape[1]).type(torch.FloatTensor)
        t = transforms.ToPILImage()(torch.cat((t,t,t),0))
        t = transforms.Resize((224,224))(t)
        t = transforms.ToTensor()(t)
        t = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(t)
        return {'image': t,
                'category': torch.from_numpy(np.array(category)).type(torch.LongTensor)} 

In [7]:
train_dataset = HieroglyphsDataset(files=['train.npy'], transform=Trans())
test_dataset = HieroglyphsDataset(files=['test.npy'], transform=Trans())

In [8]:
trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
print(len(trainloader), len(trainloader.dataset))
for i_batch, sample_batched in enumerate(trainloader):
    print(i_batch, sample_batched['image'].size(),
          sample_batched['category'].size())
    if i_batch == 3:
        break

1821 116522
0 torch.Size([64, 3, 224, 224]) torch.Size([64])
1 torch.Size([64, 3, 224, 224]) torch.Size([64])
2 torch.Size([64, 3, 224, 224]) torch.Size([64])
3 torch.Size([64, 3, 224, 224]) torch.Size([64])


In [9]:
testloader = DataLoader(test_dataset, batch_size=64, shuffle=True)
print(len(testloader), len(testloader.dataset))
for i_batch, sample_batched in enumerate(testloader):
    print(i_batch, sample_batched['image'].size(),
          sample_batched['category'].size())
    if i_batch == 3:
        break

781 49938
0 torch.Size([64, 3, 224, 224]) torch.Size([64])
1 torch.Size([64, 3, 224, 224]) torch.Size([64])
2 torch.Size([64, 3, 224, 224]) torch.Size([64])
3 torch.Size([64, 3, 224, 224]) torch.Size([64])


In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [11]:
def Train(model, train_loader, optimizer):
    model.train()
    
    processed = 0
    total = len(train_loader.dataset)
    
    for batch_idx, sample_batched in enumerate(train_loader):
        X, y = sample_batched['image'], sample_batched['category']
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        probs = net(X)
        loss = criterion(probs, y)
        if not batch_idx % 10:
            print(f'{processed}/{total}, loss: {loss.item()}')
            
        loss.backward()
        optimizer.step()      
        
        processed += len(X)

In [12]:
def Test(model, test_loader):
    with torch.no_grad():
        model.eval()
        processed = 0
        total = len(test_loader.dataset)
        sum = 0
        for batch_idx, sample_batched in enumerate(test_loader):
            X, y = sample_batched['image'], sample_batched['category']
            X, y = X.to(device), y.to(device)
            probs = net(X)
            print(probs.size())
            s = np.sum(np.array([int(probs[i].argmax().item() == y[i].item()) for i in range(len(probs))]))
            processed += len(X)
            sum += s
            if not batch_idx % 100:
                print(sum/processed)
        print(sum, '/', total, sum/total)

In [13]:
net = models.vgg16()
name = 'vgg'
net.to(device)
print()

In [14]:
net = torch.load('vgg2epochs')#5epochs

In [15]:
epoch_n = 3
net.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=1e-3, momentum=0.9)

for epoch in range(3, epoch_n+3):
    print(f'Epoch {epoch}:')
    Train(net, trainloader, optimizer)
    torch.save(net, f'{name}{epoch}epochs')
    Test(net, testloader)

Epoch 3:
0/116522, loss: 0.5163497924804688
640/116522, loss: 0.4995899796485901
1280/116522, loss: 0.4122880697250366
1920/116522, loss: 0.2210594117641449
2560/116522, loss: 0.2873203456401825
3200/116522, loss: 0.6210335493087769
3840/116522, loss: 0.37010371685028076
4480/116522, loss: 0.25982046127319336
5120/116522, loss: 0.3203766644001007
5760/116522, loss: 0.4485197365283966
6400/116522, loss: 0.5420888066291809
7040/116522, loss: 0.2786552309989929
7680/116522, loss: 0.46071481704711914
8320/116522, loss: 0.2772943079471588
8960/116522, loss: 0.35225710272789
9600/116522, loss: 0.2994958162307739
10240/116522, loss: 0.5052920579910278
10880/116522, loss: 0.3354016840457916
11520/116522, loss: 0.3998247981071472
12160/116522, loss: 0.19705317914485931
12800/116522, loss: 0.4498958885669708
13440/116522, loss: 0.49807173013687134
14080/116522, loss: 0.3650754690170288
14720/116522, loss: 0.34903472661972046
15360/116522, loss: 0.10012169182300568
16000/116522, loss: 0.230484664

13440/116522, loss: 0.15445563197135925
14080/116522, loss: 0.3494598865509033
14720/116522, loss: 0.1041504442691803
15360/116522, loss: 0.5790320634841919
16000/116522, loss: 0.1462101936340332
16640/116522, loss: 0.32962411642074585
17280/116522, loss: 0.11906042695045471
17920/116522, loss: 0.14401285350322723
18560/116522, loss: 0.19484874606132507
19200/116522, loss: 0.32206282019615173
19840/116522, loss: 0.09650209546089172
20480/116522, loss: 0.5602681040763855
21120/116522, loss: 0.36013200879096985
21760/116522, loss: 0.3405401110649109
22400/116522, loss: 0.33443182706832886
23040/116522, loss: 0.1596911996603012
23680/116522, loss: 0.3159656524658203
24320/116522, loss: 0.10855336487293243
24960/116522, loss: 0.17280186712741852
25600/116522, loss: 0.33247384428977966
26240/116522, loss: 0.16275827586650848
26880/116522, loss: 0.1568305343389511
27520/116522, loss: 0.3557543158531189
28160/116522, loss: 0.12893284857273102
28800/116522, loss: 0.3341549336910248
29440/11652

KeyboardInterrupt: 

In [16]:
Test(net, testloader)

0.921875
0.9439975247524752
0.9467506218905473
0.9470514950166113
0.9471243765586035
0.9475424151696606
0.9477173460898503
0.9470622325249644
47278 / 49938 0.9467339500981217


In [17]:
torch.save(net, f'{name}4epochs')

In [32]:
class TestDataset(Dataset):
    
    def __init__(self, files, transform=None):
        self.frame = np.load(files)
        self.transform = transform

    def __len__(self):
        return len(self.frame)

    def __getitem__(self, idx):
        sample = self.frame[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample
class Trans(object):
    def __call__(self, sample):
        image = sample
        t = torch.from_numpy(image).view(-1,image.shape[0],image.shape[1]).type(torch.FloatTensor)
        t = transforms.ToPILImage()(torch.cat((t,t,t),0))
        t = transforms.Resize((224,224))(t)
        t = transforms.ToTensor()(t)
        t = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(t)
        return t 
data_dataset = TestDataset(files='testD.npy', transform=Trans())
dataloader = DataLoader(data_dataset, batch_size=64, shuffle=True)
print(len(dataloader), len(dataloader.dataset))
for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, sample_batched.size())
    if i_batch == 3:
        break

1301 83247
0 torch.Size([64, 3, 224, 224])
1 torch.Size([64, 3, 224, 224])
2 torch.Size([64, 3, 224, 224])
3 torch.Size([64, 3, 224, 224])


In [79]:
def Pred(model, test_loader):
    preds = []
    with torch.no_grad():
        model.eval()
        processed = 0
        total = len(test_loader)
        sum = 0
        for batch_idx, sample_batched in enumerate(test_loader):
            if not batch_idx % 100:
                print(batch_idx, total)
            X = sample_batched.to(device)
            probs = net(X)
           # print(classes[np.array([int(probs[i].argmax().item()) for i in range(len(probs))])])
            arr = classes[np.array([int(probs[i].argmax().item()) for i in range(len(probs))])]
            preds = np.append(preds, arr)
           # print(preds)
    with open('preds.csv', 'w') as out:
        print('Id,Category', file=out)
        for i,c in enumerate(preds):
            print(i+1,',',c,sep='',file=out)

In [ ]:
Pred(net, dataloader)

0 1301


In [15]:
epoch_n = 5
net.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=1e-3, momentum=0.9)

for epoch in range(4, 4+epoch_n):
    print(f'Epoch {epoch}:')
    Train(net, trainloader, optimizer)
    torch.save(net, f'{name}{epoch}epochs')
    Test(net, testloader)

Epoch 4:
0/116522, loss: nan
128/116522, loss: nan
256/116522, loss: nan
384/116522, loss: nan
512/116522, loss: nan
640/116522, loss: nan
768/116522, loss: nan
896/116522, loss: nan
1024/116522, loss: nan
1152/116522, loss: nan
1280/116522, loss: nan
1408/116522, loss: nan
1536/116522, loss: nan
1664/116522, loss: nan
1792/116522, loss: nan
1920/116522, loss: nan
2048/116522, loss: nan
2176/116522, loss: nan
2304/116522, loss: nan
2432/116522, loss: nan
2560/116522, loss: nan
2688/116522, loss: nan
2816/116522, loss: nan
2944/116522, loss: nan
3072/116522, loss: nan
3200/116522, loss: nan
3328/116522, loss: nan
3456/116522, loss: nan
3584/116522, loss: nan
3712/116522, loss: nan
3840/116522, loss: nan
3968/116522, loss: nan
4096/116522, loss: nan
4224/116522, loss: nan
4352/116522, loss: nan
4480/116522, loss: nan
4608/116522, loss: nan
4736/116522, loss: nan
4864/116522, loss: nan
4992/116522, loss: nan
5120/116522, loss: nan
5248/116522, loss: nan
5376/116522, loss: nan
5504/116522,

44160/116522, loss: nan
44288/116522, loss: nan
44416/116522, loss: nan
44544/116522, loss: nan
44672/116522, loss: nan
44800/116522, loss: nan
44928/116522, loss: nan
45056/116522, loss: nan
45184/116522, loss: nan
45312/116522, loss: nan
45440/116522, loss: nan
45568/116522, loss: nan
45696/116522, loss: nan
45824/116522, loss: nan
45952/116522, loss: nan
46080/116522, loss: nan
46208/116522, loss: nan
46336/116522, loss: nan
46464/116522, loss: nan
46592/116522, loss: nan
46720/116522, loss: nan
46848/116522, loss: nan
46976/116522, loss: nan
47104/116522, loss: nan
47232/116522, loss: nan
47360/116522, loss: nan
47488/116522, loss: nan
47616/116522, loss: nan
47744/116522, loss: nan
47872/116522, loss: nan
48000/116522, loss: nan
48128/116522, loss: nan
48256/116522, loss: nan
48384/116522, loss: nan
48512/116522, loss: nan
48640/116522, loss: nan
48768/116522, loss: nan
48896/116522, loss: nan
49024/116522, loss: nan
49152/116522, loss: nan
49280/116522, loss: nan
49408/116522, lo

87936/116522, loss: nan
88064/116522, loss: nan
88192/116522, loss: nan
88320/116522, loss: nan
88448/116522, loss: nan
88576/116522, loss: nan
88704/116522, loss: nan
88832/116522, loss: nan
88960/116522, loss: nan
89088/116522, loss: nan
89216/116522, loss: nan
89344/116522, loss: nan
89472/116522, loss: nan
89600/116522, loss: nan
89728/116522, loss: nan
89856/116522, loss: nan
89984/116522, loss: nan
90112/116522, loss: nan
90240/116522, loss: nan
90368/116522, loss: nan
90496/116522, loss: nan
90624/116522, loss: nan
90752/116522, loss: nan
90880/116522, loss: nan
91008/116522, loss: nan
91136/116522, loss: nan
91264/116522, loss: nan
91392/116522, loss: nan
91520/116522, loss: nan
91648/116522, loss: nan
91776/116522, loss: nan
91904/116522, loss: nan
92032/116522, loss: nan
92160/116522, loss: nan
92288/116522, loss: nan
92416/116522, loss: nan
92544/116522, loss: nan
92672/116522, loss: nan
92800/116522, loss: nan
92928/116522, loss: nan
93056/116522, loss: nan
93184/116522, lo

13824/116522, loss: nan
13952/116522, loss: nan
14080/116522, loss: nan
14208/116522, loss: nan
14336/116522, loss: nan
14464/116522, loss: nan
14592/116522, loss: nan
14720/116522, loss: nan
14848/116522, loss: nan
14976/116522, loss: nan
15104/116522, loss: nan
15232/116522, loss: nan
15360/116522, loss: nan
15488/116522, loss: nan
15616/116522, loss: nan
15744/116522, loss: nan
15872/116522, loss: nan
16000/116522, loss: nan
16128/116522, loss: nan
16256/116522, loss: nan
16384/116522, loss: nan
16512/116522, loss: nan
16640/116522, loss: nan
16768/116522, loss: nan
16896/116522, loss: nan
17024/116522, loss: nan
17152/116522, loss: nan
17280/116522, loss: nan
17408/116522, loss: nan
17536/116522, loss: nan
17664/116522, loss: nan
17792/116522, loss: nan
17920/116522, loss: nan
18048/116522, loss: nan
18176/116522, loss: nan
18304/116522, loss: nan
18432/116522, loss: nan
18560/116522, loss: nan
18688/116522, loss: nan
18816/116522, loss: nan
18944/116522, loss: nan
19072/116522, lo

57600/116522, loss: nan
57728/116522, loss: nan
57856/116522, loss: nan
57984/116522, loss: nan
58112/116522, loss: nan
58240/116522, loss: nan
58368/116522, loss: nan
58496/116522, loss: nan
58624/116522, loss: nan
58752/116522, loss: nan
58880/116522, loss: nan
59008/116522, loss: nan
59136/116522, loss: nan
59264/116522, loss: nan
59392/116522, loss: nan
59520/116522, loss: nan
59648/116522, loss: nan
59776/116522, loss: nan
59904/116522, loss: nan
60032/116522, loss: nan
60160/116522, loss: nan
60288/116522, loss: nan
60416/116522, loss: nan
60544/116522, loss: nan
60672/116522, loss: nan
60800/116522, loss: nan
60928/116522, loss: nan
61056/116522, loss: nan
61184/116522, loss: nan
61312/116522, loss: nan
61440/116522, loss: nan
61568/116522, loss: nan
61696/116522, loss: nan
61824/116522, loss: nan
61952/116522, loss: nan
62080/116522, loss: nan
62208/116522, loss: nan
62336/116522, loss: nan
62464/116522, loss: nan
62592/116522, loss: nan
62720/116522, loss: nan
62848/116522, lo

101248/116522, loss: nan
101376/116522, loss: nan
101504/116522, loss: nan
101632/116522, loss: nan
101760/116522, loss: nan
101888/116522, loss: nan
102016/116522, loss: nan
102144/116522, loss: nan
102272/116522, loss: nan
102400/116522, loss: nan
102528/116522, loss: nan
102656/116522, loss: nan
102784/116522, loss: nan
102912/116522, loss: nan
103040/116522, loss: nan
103168/116522, loss: nan
103296/116522, loss: nan
103424/116522, loss: nan
103552/116522, loss: nan
103680/116522, loss: nan
103808/116522, loss: nan
103936/116522, loss: nan
104064/116522, loss: nan
104192/116522, loss: nan
104320/116522, loss: nan
104448/116522, loss: nan
104576/116522, loss: nan
104704/116522, loss: nan
104832/116522, loss: nan
104960/116522, loss: nan
105088/116522, loss: nan
105216/116522, loss: nan
105344/116522, loss: nan
105472/116522, loss: nan
105600/116522, loss: nan
105728/116522, loss: nan
105856/116522, loss: nan
105984/116522, loss: nan
106112/116522, loss: nan
106240/116522, loss: nan


27136/116522, loss: nan
27264/116522, loss: nan
27392/116522, loss: nan
27520/116522, loss: nan
27648/116522, loss: nan
27776/116522, loss: nan
27904/116522, loss: nan
28032/116522, loss: nan
28160/116522, loss: nan
28288/116522, loss: nan
28416/116522, loss: nan
28544/116522, loss: nan
28672/116522, loss: nan
28800/116522, loss: nan
28928/116522, loss: nan
29056/116522, loss: nan
29184/116522, loss: nan
29312/116522, loss: nan
29440/116522, loss: nan
29568/116522, loss: nan
29696/116522, loss: nan
29824/116522, loss: nan
29952/116522, loss: nan
30080/116522, loss: nan
30208/116522, loss: nan
30336/116522, loss: nan
30464/116522, loss: nan
30592/116522, loss: nan
30720/116522, loss: nan
30848/116522, loss: nan
30976/116522, loss: nan
31104/116522, loss: nan
31232/116522, loss: nan
31360/116522, loss: nan
31488/116522, loss: nan
31616/116522, loss: nan
31744/116522, loss: nan
31872/116522, loss: nan
32000/116522, loss: nan
32128/116522, loss: nan
32256/116522, loss: nan
32384/116522, lo

70912/116522, loss: nan
71040/116522, loss: nan
71168/116522, loss: nan
71296/116522, loss: nan
71424/116522, loss: nan
71552/116522, loss: nan
71680/116522, loss: nan
71808/116522, loss: nan
71936/116522, loss: nan
72064/116522, loss: nan
72192/116522, loss: nan
72320/116522, loss: nan
72448/116522, loss: nan
72576/116522, loss: nan
72704/116522, loss: nan
72832/116522, loss: nan
72960/116522, loss: nan
73088/116522, loss: nan
73216/116522, loss: nan
73344/116522, loss: nan
73472/116522, loss: nan
73600/116522, loss: nan
73728/116522, loss: nan
73856/116522, loss: nan
73984/116522, loss: nan
74112/116522, loss: nan
74240/116522, loss: nan
74368/116522, loss: nan
74496/116522, loss: nan
74624/116522, loss: nan
74752/116522, loss: nan
74880/116522, loss: nan
75008/116522, loss: nan
75136/116522, loss: nan
75264/116522, loss: nan
75392/116522, loss: nan
75520/116522, loss: nan
75648/116522, loss: nan
75776/116522, loss: nan
75904/116522, loss: nan
76032/116522, loss: nan
76160/116522, lo

KeyboardInterrupt: 

In [ ]:
epoch_n = 5
net.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=1e-3, momentum=0.9)

for epoch in range(5, 5+epoch_n):
    print(f'Epoch {epoch}:')
    Train(net, trainloader, optimizer)
    torch.save(net, f'{name}{epoch}epochs')
    Test(net, testloader)

In [ ]:
0.948